## Predicting Parole Violators

In many criminal justice systems around the world, inmates deemed not to be a threat to society are released from prison under the parole system prior to completing their sentence. They are still considered to be serving their sentence while on parole, and they can be returned to prison if they violate the terms of their parole.

Parole boards are charged with identifying which inmates are good candidates for release on parole. They seek to release inmates who will not commit additional crimes after release. In this problem, we will build and validate a model that predicts if an inmate will violate the terms of his or her parole. Such a model could be useful to a parole board when deciding to approve or deny an application for parole.

For this prediction task, we will use data from the United States 2004 National Corrections Reporting Program, a nationwide census of parole releases that occurred during 2004. We limited our focus to parolees who served no more than 6 months in prison and whose maximum sentence for all charges did not exceed 18 months. The dataset contains all such parolees who either successfully completed their term of parole during 2004 or those who violated the terms of their parole during that year.

The dataset contains the following variables:

- male: 1 if the parolee is male, 0 if female
- race: 1 if the parolee is white, 2 otherwise
- age: the parolee's age (in years) when he or she was released from prison
- state: a code for the parolee's state. 2 is Kentucky, 3 is Louisiana, 4 is Virginia, and 1 is any other state. The three states were selected due to having a high representation in the dataset.
- time.served: the number of months the parolee served in prison (limited by the inclusion criteria to not exceed 6 months).
- max.sentence: the maximum sentence length for all charges, in months (limited by the inclusion criteria to not exceed 18 months).
- multiple.offenses: 1 if the parolee was incarcerated for multiple offenses, 0 otherwise.
- crime: a code for the parolee's main crime leading to incarceration. 2 is larceny, 3 is drug-related crime, 4 is driving-related crime, and 1 is any other crime.
- violator: 1 if the parolee violated the parole, and 0 if the parolee completed the parole without violation.

### Loading the Dataset

In [1]:
parole = read.csv('./dataset/parole.csv')
str(parole)

'data.frame':	675 obs. of  9 variables:
 $ male             : int  1 0 1 1 1 1 1 0 0 1 ...
 $ race             : int  1 1 2 1 2 2 1 1 1 2 ...
 $ age              : num  33.2 39.7 29.5 22.4 21.6 46.7 31 24.6 32.6 29.1 ...
 $ state            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ time.served      : num  5.5 5.4 5.6 5.7 5.4 6 6 4.8 4.5 4.7 ...
 $ max.sentence     : int  18 12 12 18 12 18 18 12 13 12 ...
 $ multiple.offenses: int  0 0 0 0 0 0 0 0 0 0 ...
 $ crime            : int  4 3 3 1 1 4 3 1 3 2 ...
 $ violator         : int  0 0 0 0 0 0 0 0 0 0 ...


In [3]:
table(parole$violator)


  0   1 
597  78 

In [4]:
summary(parole)

      male             race            age            state      
 Min.   :0.0000   Min.   :1.000   Min.   :18.40   Min.   :1.000  
 1st Qu.:1.0000   1st Qu.:1.000   1st Qu.:25.35   1st Qu.:2.000  
 Median :1.0000   Median :1.000   Median :33.70   Median :3.000  
 Mean   :0.8074   Mean   :1.424   Mean   :34.51   Mean   :2.887  
 3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:42.55   3rd Qu.:4.000  
 Max.   :1.0000   Max.   :2.000   Max.   :67.00   Max.   :4.000  
  time.served     max.sentence   multiple.offenses     crime      
 Min.   :0.000   Min.   : 1.00   Min.   :0.0000    Min.   :1.000  
 1st Qu.:3.250   1st Qu.:12.00   1st Qu.:0.0000    1st Qu.:1.000  
 Median :4.400   Median :12.00   Median :1.0000    Median :2.000  
 Mean   :4.198   Mean   :13.06   Mean   :0.5363    Mean   :2.059  
 3rd Qu.:5.200   3rd Qu.:15.00   3rd Qu.:1.0000    3rd Qu.:3.000  
 Max.   :6.000   Max.   :18.00   Max.   :1.0000    Max.   :4.000  
    violator     
 Min.   :0.0000  
 1st Qu.:0.0000  
 Median :0.0000

In [7]:
unique(parole$crime)

[1] 4 3 1 2

In the last subproblem, we identified variables that are unordered factors with at least 3 levels, so we need to convert them to factors for our prediction problem (we introduced this idea in the "Reading Test Scores" problem last week). Using the as.factor() function, convert these variables to factors. Keep in mind that we are not changing the values, just the way R understands them (the values are still numbers).

In [11]:
parole$state = as.factor(parole$state)
parole$crime = as.factor(parole$crime)

In [12]:
summary(parole)

      male             race            age        state    time.served   
 Min.   :0.0000   Min.   :1.000   Min.   :18.40   1:143   Min.   :0.000  
 1st Qu.:1.0000   1st Qu.:1.000   1st Qu.:25.35   2:120   1st Qu.:3.250  
 Median :1.0000   Median :1.000   Median :33.70   3: 82   Median :4.400  
 Mean   :0.8074   Mean   :1.424   Mean   :34.51   4:330   Mean   :4.198  
 3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:42.55           3rd Qu.:5.200  
 Max.   :1.0000   Max.   :2.000   Max.   :67.00           Max.   :6.000  
  max.sentence   multiple.offenses crime      violator     
 Min.   : 1.00   Min.   :0.0000    1:315   Min.   :0.0000  
 1st Qu.:12.00   1st Qu.:0.0000    2:106   1st Qu.:0.0000  
 Median :12.00   Median :1.0000    3:153   Median :0.0000  
 Mean   :13.06   Mean   :0.5363    4:101   Mean   :0.1156  
 3rd Qu.:15.00   3rd Qu.:1.0000            3rd Qu.:0.0000  
 Max.   :18.00   Max.   :1.0000            Max.   :1.0000  

In [13]:
str(parole)

'data.frame':	675 obs. of  9 variables:
 $ male             : int  1 0 1 1 1 1 1 0 0 1 ...
 $ race             : int  1 1 2 1 2 2 1 1 1 2 ...
 $ age              : num  33.2 39.7 29.5 22.4 21.6 46.7 31 24.6 32.6 29.1 ...
 $ state            : Factor w/ 4 levels "1","2","3","4": 1 1 1 1 1 1 1 1 1 1 ...
 $ time.served      : num  5.5 5.4 5.6 5.7 5.4 6 6 4.8 4.5 4.7 ...
 $ max.sentence     : int  18 12 12 18 12 18 18 12 13 12 ...
 $ multiple.offenses: int  0 0 0 0 0 0 0 0 0 0 ...
 $ crime            : Factor w/ 4 levels "1","2","3","4": 4 3 3 1 1 4 3 1 3 2 ...
 $ violator         : int  0 0 0 0 0 0 0 0 0 0 ...


### Splitting into a Training and Testing Set

In [14]:
set.seed(144)
library(caTools)
split = sample.split(parole$violator, SplitRatio = 0.7)
train = subset(parole, split == TRUE)
test = subset(parole, split == FALSE)

### Building a Logistic Regression Model

In [15]:
violatorReg = glm(violator ~ ., data=train, family='binomial')
summary(violatorReg)


Call:
glm(formula = violator ~ ., family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7041  -0.4236  -0.2719  -0.1690   2.8375  

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)       -4.2411574  1.2938852  -3.278  0.00105 ** 
male               0.3869904  0.4379613   0.884  0.37690    
race               0.8867192  0.3950660   2.244  0.02480 *  
age               -0.0001756  0.0160852  -0.011  0.99129    
state2             0.4433007  0.4816619   0.920  0.35739    
state3             0.8349797  0.5562704   1.501  0.13335    
state4            -3.3967878  0.6115860  -5.554 2.79e-08 ***
time.served       -0.1238867  0.1204230  -1.029  0.30359    
max.sentence       0.0802954  0.0553747   1.450  0.14705    
multiple.offenses  1.6119919  0.3853050   4.184 2.87e-05 ***
crime2             0.6837143  0.5003550   1.366  0.17180    
crime3            -0.2781054  0.4328356  -0.643  0.52054    
crime4  

In [16]:
exp(1.61)

[1] 5.002811

In [47]:
-4.2411574 + 0.3869904 * 1 + 0.8867192 * 1 + -0.0001756 * 50 + -0.1238867 * 3 + 0.0802954 * 12 + 0.6837143

[1] -1.700629

In [52]:
exp(-1.7006288)

[1] 0.1825687

In [48]:
1 / (1 + exp(1.7006288))

[1] 0.1543832

### Evaluating the Model on the Testing Set

In [53]:
testPred = predict(violatorReg, newdata=test, type='response')
max(testPred)

[1] 0.9072791

In [54]:
table(test$violator, testPred >= 0.5)

   
    FALSE TRUE
  0   167   12
  1    11   12

In [55]:
sensitivity = 12 / (11 + 12)
specificity = 167 / (167 + 12)
accuracy = (167 + 12) / (167 + 12 + 11 + 11)
sensitivity
specificity
accuracy

[1] 0.5217391

[1] 0.9329609

[1] 0.8905473

In [56]:
non_violator= subset(test, test$violator == 0)
nrow(non_violator) / nrow(test)

[1] 0.8861386

In [60]:
table(test$violator, testPred >= 0.25)

   
    FALSE TRUE
  0   156   23
  1     7   16

In [61]:
library("ROCR")

Loading required package: gplots


Attaching package: ‘gplots’


The following object is masked from ‘package:stats’:

    lowess




In [62]:
ROCRpredTest = prediction(testPred, test$violator)
ROCRpredPerf = performance(ROCRpredTest, "auc")@y.values
auc =as.numeric(ROCRpredPerf)

In [63]:
auc

[1] 0.8945834